In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

In [45]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [46]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

# 1. Data

In [47]:
data_fake = pd.read_csv("../raw_data/fake-news.csv")

In [56]:
data_fake_sub = data_fake.head(200)

In [57]:
X = data_fake_sub['text']
y = data_fake_sub['label']

In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 1. Preprocessor

The model we chose for BERT Embedding works with a specific preprocessor  
it inlcudes tokenization and padding

In [59]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

2023-06-07 17:42:39.239869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-07 17:42:39.240445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-07 17:42:39.240631: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-06-07 17:42:39.241103: I tensorflow/core/

In [60]:
#text_test = ['ok I see what you mean darling', 'do not do that in the desert or you will die']
X_train_preproc = bert_preprocess_model(X_train)  # it is a dict
print(X_train_preproc.keys())

# input_word_ids : contains the IDs of each token in the input text
# input_mask: indicates which tokens in the input sequence are actual words and which ones are padding tokens
# => tensor of a certain shape, shape[0] is the number of sentences and shape[1] length of padding
# input_type_ids

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])


# 2. BERT Encoder

In [61]:
bert_encoder_model = hub.KerasLayer(encoder_url)

In [66]:
output = bert_encoder_model(X_train_preproc)

In [67]:
output.keys()

dict_keys(['pooled_output', 'sequence_output', 'encoder_outputs', 'default'])

In [65]:
print(output['pooled_output'].shape) # => num of senteces/rows, size of embedding vector
print(len(output['encoder_outputs'])) # => 12 because we used a small BERT model with 12 encoders
print((output['encoder_outputs'][0].shape)) # => num of senteces/rows, padding size, size of embedding vector

(150, 768)
12
(150, 128, 768)


In [69]:
X_train_encoded = output['pooled_output']
X_train_encoded

<tf.Tensor: shape=(150, 768), dtype=float32, numpy=
array([[-0.7815599 , -0.41604653, -0.6697939 , ..., -0.48963648,
        -0.5472172 ,  0.80348665],
       [-0.60785097, -0.28559744, -0.6791864 , ..., -0.31198895,
        -0.56930023,  0.4917608 ],
       [-0.8159721 , -0.55098385, -0.7703171 , ..., -0.31313154,
        -0.64943296,  0.5353548 ],
       ...,
       [-0.94676465, -0.44679666, -0.77216285, ..., -0.19994205,
        -0.60622716,  0.87176985],
       [-0.81957006, -0.45411915, -0.6177753 , ..., -0.3751994 ,
        -0.6901966 ,  0.81343234],
       [-0.7721559 , -0.3242671 , -0.5379801 , ..., -0.02808042,
        -0.5813943 ,  0.50213766]], dtype=float32)>